In [3]:
import pandas as pd
import numpy as np
import pvlib
from pvlib.location import Location
import sympy as sm
import matplotlib.dates as mdates
from dtumathtools import *

from scipy import integrate

def inner(x1: sm.Matrix,x2: sm.Matrix):
    '''
    Computes the inner product of two vectors of same length.
    '''

    return x1.dot(x2, conjugate_convention = 'right')

#generation of date strings for all dates in year 2024
dateList= []
daysInMonth=[31,29,31,30,31,30,31,31,30,31,30,31]
#create list of all days
for i in range (12): #months
    for j in range (1,daysInMonth[i]+1):
        date = "2024-"
        date=date+"0"+str(i+1)+"-" if i+1<10 else date+str(i+1)+"-"#add month
        date = date +"0"+str(j) if (j<10) else date+str(j)        
        dateList.append(date)

sm.MutableDenseMatrix.inner = inner
sm.ImmutableDenseMatrix.inner = inner
L = 1.755
B = 1.038
maxFlux = 1000
maxEffect = 365/(L*B)
A = 0.5
S0 = 1100
S0A = S0 * A
effectPerFlux = maxEffect/maxFlux

def solar_elevation_angledeg(angle):
    return 90-angle

#return i, where list[i] has one sign, list[i+1] has another
def signChange(list): #return index in list
    signList=np.sign(list)
    diffList=np.diff(signList)
    switch=np.where(diffList != 0, 1,0)   
    indices = np.arange(len(switch))
    indices = indices[switch==1]
    
    return indices
def signChangeFunc(inputParameter, list): #return input value
    signList=np.sign(list)
    diffList=np.diff(signList)
    switch=np.where(diffList != 0, 1,0)   
    indices = np.arange(len(switch))
    indices = indices[switch==1]
    return inputParameter[indices]

testArray =np.array([3, 0.5, 0.1,-0.5, -1, -0.5, 0.5, 2])

display(signChange(testArray))

def solar_panel_projection(θ_s, ϕ_s, θ_p, ϕ_p):
    results = np.zeros(len(θ_s))
    for i in range(0,len(θ_s)):
        us = [-np.sin(θ_s[i]) * np.cos(ϕ_s[i]),-np.sin(θ_s[i]) * np.sin(ϕ_s[i]), -np.cos(θ_s[i])]
        up = [-1*np.sin(θ_p[i]) * np.cos(ϕ_p[i]), -np.sin(ϕ_p[i])*np.sin(θ_p[i]), -np.cos(θ_p[i])]
        inprod = (np.inner(up,us))
        results[i]=inprod
    results=results[results>0]
    return results



array([2, 5])

In [4]:

def kWhHour(zenithangs,azimuthangs, theta_p, phi_p):
    t= np.arange(len(zenithangs))
    indexOfInterest = t[zenithangs[t]>=0]
    indexOfInterest = indexOfInterest[zenithangs[indexOfInterest]<=np.pi/2]
    zenithangs=zenithangs[indexOfInterest]
    azimuthangs=azimuthangs[indexOfInterest]
    
    theta_list= np.full(len(zenithangs),theta_p)
    phi_list=np.full(len(zenithangs),phi_p)
    
    projList =  solar_panel_projection(zenithangs,azimuthangs,theta_list,phi_list)
    projList =  projList*effectPerFlux*S0A
    projList[projList>maxEffect]=maxEffect    
    return ((integrate.simpson(projList,dx = 3600))*L*B)/3600000

def kWhMin(zenithangs,azimuthangs, theta_p, phi_p):
    t= np.arange(len(zenithangs))
    indexOfInterest = t[zenithangs[t]>=0]
    indexOfInterest = indexOfInterest[zenithangs[indexOfInterest]<=np.pi/2]
    zenithangs=zenithangs[indexOfInterest]
    azimuthangs=azimuthangs[indexOfInterest]
    
    theta_list= np.full(len(zenithangs),theta_p)
    phi_list=np.full(len(zenithangs),phi_p)
    
    projList =  solar_panel_projection(zenithangs,azimuthangs,theta_list,phi_list)
    projList =  projList*effectPerFlux*S0A
    projList[projList>maxEffect]=maxEffect    
    return ((integrate.simpson(projList,dx = 60))*L*B)/3600000

In [16]:
tidszone = "Europe/Copenhagen"
start_dato = "2024-01-01"
slut_dato = "2024-12-31"
delta_tid = "H" # "Min", "H",
# Definition of Location object. Coordinates and elevation of DTU,
#Copenhagen (Denmark)
site = Location(
55.7861111111, 12.5230555556, tidszone, 10, "DTU (DK)"
) # latitude, longitude, time_zone, altitude, name
# Definition of a time range of simulation
times = pd.date_range(
start_dato + " 00:00:00", slut_dato + " 23:59:00",
inclusive="left", freq=delta_tid, tz=tidszone
)
# Estimate Solar Position with the ’Location’ object
solpos = site.get_solarposition(times)
results1 = []
#Lister af solens Zenith og Azimuth vinkler hver time over et ar
zenithangs = np.deg2rad(np.array(solpos.loc[start_dato:slut_dato].apparent_zenith))
azimuthangs = np.deg2rad(np.array(solpos.loc[start_dato:slut_dato].azimuth))
# Azimuth vinkel for panelet
phi_p = 180
#Der itereres over panelets azimuth vinkler fra 90 til 270
for j in range(170,190):
    phi_p = j
    results = []
# Der itereres over panelets zenith vinkeler fra 0 til 90
    for i in range(0,91):
        heta_p = i
# Effekten for et helt ar ved en given zenith og azimuth vinkel for panelet beregnes
        energyPrYear = kWhHour(zenithangs,azimuthangs,
        np.deg2rad(heta_p), np.deg2rad(phi_p))
        results.append(energyPrYear)
    results1.append(results)
# Finder maks effekt og tilsvarende vinkler
max = 0
angle_p=-1
angle_t=-1
for i in range(0,len(results1)):
    for j in range(0,len(results1[i])):
        if(results1[i][j] > max):
            max = results1[i][j]
            angle_p = i
            angle_t = j
print("Energy Production is:",max,"Angle is:",solar_elevation_angledeg(angle_t),
"Degrees from horizon","Azimuth degree:",angle_p+170)

Energy Production is: 515.7082259499622 Angle is: 39 Degrees from horizon Azimuth degree: 181


In [12]:
for i in range(0,len(results1)):
    for j in range(0,len(results1[i])):
        if(results1[i][j] > max):
            max = results[i]
            angle = i
            angle1 = j
        
print("Energy Production is:",max,
"Angle is:",solar_elevation_angledeg(angle1),"Degrees from horizon", angle1,)

Energy Production is: 439.7635535642131 Angle is: 8 Degrees from horizon 82
